In [15]:
# To visualize the data, we will use the pymatviz package
# ! pip install pymatviz

In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval

from pymatgen.core import Structure
from pymatviz import StructureWidget

In [2]:
data = pd.read_csv('/home/user_wanglei/private/datafile/crystalgpt/checkpoint/alex20/output_160_struct.csv')
cif_strings = data['cif']   # string of pymatgen structure dict



In [3]:
structures =[Structure.from_dict(literal_eval(cif)) for cif in cif_strings]

In [4]:
len(structures)

1000

In [8]:
StructureWidget(structures[10])

StructureWidget(structure={'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattic…